In [1]:
import numpy as np
from scipy.linalg import svd

## Syntax:
U, s, Vh = scipy.linalg.svd(A, full_matrices=True, compute_uv=True)

- A: The input matrix (as a NumPy array) to be decomposed.

- full_matrices (optional, boolean): If True (default), U and Vh have shapes (M, M) and (N, N). If False, the shapes are (M, K) and (K, N), where K = min(M, N). Using full_matrices=False is more memory-efficient and is often called "thin SVD".

- compute_uv (optional, boolean): If True (default), the full U and Vh matrices are computed. If False, only the singular values s are computed.

In [2]:
# Create a 3x2 matrix
A = np.array([[1, 2], [3, 4], [5, 6]])

# Decompose the matrix A
U, s, Vh = svd(A)

print("\nU (Left Singular Vectors):")
print(U)

print("\ns (Singular Values as a 1D array):")
print(s)

print("\nVh (Transposed Right Singular Vectors):")
print(Vh)


U (Left Singular Vectors):
[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]

s (Singular Values as a 1D array):
[9.52551809 0.51430058]

Vh (Transposed Right Singular Vectors):
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


Note: singular values are returned as 1D array and not matrix.

## Reconstruction of original Matrix
1. Create zero matrix
2. fill diagonal with singular values
3. perform matrix multiplication 

In [3]:
# step 1
Sigma = np.zeros(A.shape)
print("Empty Sigma matrix:", "\n", Sigma)

# step 2, can't use Sigma = np.diag(s), as Sigma is not a square matrix
Sigma[: A.shape[1], : A.shape[1]] = np.diag(s)

# step 3, @ operator is used for matrix multiplication
A_recon = U @ Sigma @ Vh
print("Recostructed A", "\n", A_recon)

assert np.allclose(A, A_recon)
print("\nVerification successful: Reconstructed matrix is close to the original.")

Empty Sigma matrix: 
 [[0. 0.]
 [0. 0.]
 [0. 0.]]
Recostructed A 
 [[1. 2.]
 [3. 4.]
 [5. 6.]]

Verification successful: Reconstructed matrix is close to the original.


## Truncated SVD for Approximation
 key application of SVD is approximating a matrix by using only the top k singular values. This is the basis for techniques like Principal Component Analysis (PCA) and data compression.

In [4]:
# Number of components to keep
k = 1

# Truncate the matrices
U_trunc = U[:, :k]
Sigma_trunc = np.diag(s[:k])
Vh_trunc = Vh[:k, :]

# Reconstruct the approximated matrix
A_approx = U_trunc @ Sigma_trunc @ Vh_trunc

print(f"\nMatrix Approximated with k={k} singular value(s):")
print(A_approx)


Matrix Approximated with k=1 singular value(s):
[[1.35662819 1.71846235]
 [3.09719707 3.92326845]
 [4.83776596 6.12807454]]


# Turning any state into MPOs    

In [5]:
n = 3 # three sites = three legs

# set up random tensor and normalise it
psi = np.random.rand(2**3)
# print("Psi before reshaping: \n ", psi, "\n ")
psi = psi / np.linalg.norm(psi)  # random, normalized state vector
psi = np.reshape(psi, (2, 2, 2)) # rewrite psi as rank-n tensor
print("Psi normalised and reshaped into 2x2x2 tensor: \n", psi)

#                      - SITE 1 -
# Step 1 - reshape tensor to matrix, required for svd 
psi = np.reshape(psi, (2, 2**(n-1)))
print("Reshape tensor as matrix : \n ", psi)

# Step 2 - SVD to split off first site
U, Lambda, Vd = svd(psi, full_matrices=False)
# print("Shape of U right after SVD: \n", U.shape)

Us = []
# Step 3.5 - add left dummy left virtual index
U = np.reshape(U, (1, 2, 2)) # dummy_mu0, s1, mu1
# print("Shape of U after reshape: \n", U.shape)
Us.append(U)

Psi normalised and reshaped into 2x2x2 tensor: 
 [[[0.45218879 0.55005769]
  [0.00554793 0.42789196]]

 [[0.40719939 0.37914551]
  [0.00221827 0.0164909 ]]]
Reshape tensor as matrix : 
  [[0.45218879 0.55005769 0.00554793 0.42789196]
 [0.40719939 0.37914551 0.00221827 0.0164909 ]]


In [6]:

# Step 3 - Compute the remainder state psi', diagonalise Lambda first since its a 1D array 
psi1_remainder = np.diag(Lambda) @ Vd                 # mu1, (s2 s3)     =>  this has shape (2,4) for a 3 qubits system
print("Psi' remainder", psi1_remainder.shape, psi1_remainder)

#                      - SITE 2 -
# Step 4 - Reshaping, fuse mu1 and s2, this will separate s2 and s3
psi1_remainder = np.reshape(psi1_remainder, (4, 2))  # (mu1 s2), s3   =>  this has shape (4,2) 
print("Psi' remainder reshaped:", psi1_remainder.shape, psi1_remainder)
U, Lambda, Vd = svd(psi1_remainder, full_matrices = False)  # U has shape 4,2

U = np.reshape(U, (2, 2, 2)) # mu1, s2, mu2
Us.append(U)

print("U shape:", U.shape, "Lamda shape:", Lambda.shape,"Vd shape:", Vd.shape)

Psi' remainder (2, 4) [[-0.59976979 -0.66753069 -0.00587518 -0.37056813]
 [ 0.10277278  0.02678723 -0.00108746 -0.21457571]]
Psi' remainder reshaped: (4, 2) [[-0.59976979 -0.66753069]
 [-0.00587518 -0.37056813]
 [ 0.10277278  0.02678723]
 [-0.00108746 -0.21457571]]
U shape: (2, 2, 2) Lamda shape: (2,) Vd shape: (2, 2)


In [ ]:
#                      - SITE 3 -
# Step 5 - Compute remainder psi''
psi2_remainder = np.diag(Lambda) @ Vd                 # mu2, s3   
print("Psi'' remainder reshaped:", psi2_remainder.shape, psi2_remainder)

Psi'' remainder reshaped: (2, 2) [[ 0.56735715  0.77708658]
 [-0.22006274  0.16066957]]


Because our state vector was already normalized, the singular value in this last SVD is just 1. You could just reshape this matrix into the final tensor (mu2, s3, 1) (hence add additional virtual index) and be done.

However, what if the original state was not normalized? All the "norm" of the state has been pushed into this final matrix. (a good exercise to confirm by skipping the normalization step in the definition of psi above).

In [12]:
def norm_checker(Lamda_tmp):
    "Controls if the state is normalised by checking if Lamda is 1, if not it does another SVD to handle the norm"

    if np.allclose(Lamda_tmp, 1):
        print("Lambda is 1, hence the state is normalized.")
        # add right dummy index
        U = np.reshape(psi2_remainder, (2, 2, 1)) # Reshape to (mu2, s3, dummy_mu3)
        Us.append(U)
    else:
        print("Lambda is not 1, initial state was not normalized. Another SVD was performed to handle the norm.")
        # for programmatic elegance and to handle normalization, we do another SVD 
        # psi2_remainder = np.reshape(psi2_remainder, (4, 1))  # (mu1 s2), s3 
        U, Lambda, Vd = svd(psi2_remainder, full_matrices=False)

        # Add right dummy index
        # U = np.reshape(U, (2, 2, 1)) # Reshape to (mu2, s3, dummy_mu3)
        Us.append(U)

        print(U.shape, Lambda.shape, Vd.shape)

norm_checker(Lambda)

Lambda is not 1, initial state was not normalized. Another SVD was performed to handle the norm.
(2, 2) (2,) (2, 2)


In [ ]:
print(f"Shapes of Us: {[_.shape for _ in Us]}")

psi_reconstruct = Us[0]

for i in range(1, len(Us)):
    # contract the rightmost with the left most index
    psi_reconstruct = np.tensordot(psi_reconstruct, Us[i], axes=1)

print(f"Shape of reconstructed psi: {psi_reconstruct.shape}")
# remove dummy dimensions
psi_reconstruct = np.reshape(psi_reconstruct, (2, 2, 2))
# original shape of original psi
psi = np.reshape(psi, (2, 2, 2))

np.allclose(psi, psi_reconstruct)

Shapes of Us: [(1, 2, 2), (2, 2, 2), (2, 2)]
Shape of reconstructed psi: (1, 2, 2, 2)


False

In [ ]:

# def contract_mpo_pair(W_left, W_right):
#     """
#     Contracts two MPO tensors together.

#     This function performs matrix multiplication on the bond dimensions
#     and the Kronecker (tensor) product on the physical dimensions.

#     Args:
#         W_left: The left MPO tensor with shape (bond_l, bond_mid, phys, phys).
#         W_right: The right MPO tensor with shape (bond_mid, bond_r, phys, phys).

#     Returns:
#         The resulting contracted tensor with shape (bond_l, bond_r, phys*phys, phys*phys).
#     """
#     # Get the dimensions
#     bond_l, bond_mid, phys, _ = W_left.shape
#     _, bond_r, _, _ = W_right.shape
    
#     # The new physical dimension will be the product of the old ones
#     new_phys_dim = phys * phys
    
#     # Initialize the result tensor
#     result = np.zeros((bond_l, bond_r, new_phys_dim, new_phys_dim), dtype=complex)
    
#     # Sum over the shared middle bond index
#     for i in range(bond_l):
#         for j in range(bond_r):
#             # Temp matrix to store the sum over the middle bond
#             sum_matrix = np.zeros((new_phys_dim, new_phys_dim), dtype=complex)
#             for k in range(bond_mid):
#                 # The core operation: Kronecker product for physical indices
#                 op_left = W_left[i, k]
#                 op_right = W_right[k, j]
#                 sum_matrix += np.kron(op_left, op_right)
#             result[i, j] = sum_matrix
            
#     return result


In [ ]:

# # --- 1. Define the Basic Operators (2x2 matrices) ---
# I = np.identity(2, dtype=complex)
# sz = np.array([[1, 0], [0, -1]], dtype=complex)
# Z = np.zeros((2, 2), dtype=complex) # Zero operator for off-diagonal blocks

# # --- 2. Construct the MPO Tensors ---
# # Each tensor W has shape (bond_left, bond_right, physical, physical)

# # W1: Shape (1, 3, 2, 2)
# W1_list = [[sz, I, I]]
# W1 = np.array(W1_list)

# # W2: Shape (3, 3, 2, 2)
# W2_list = [[sz, Z,  Z ],
#            [Z,  sz, Z ],
#            [Z,  Z,  I ]]
# W2 = np.array(W2_list)

# # W3: Shape (3, 3, 2, 2)
# W3_list = [[I,  Z,  Z ],
#            [Z,  sz, Z ],
#            [Z,  Z,  sz]]
# W3 = np.array(W3_list)

# # W4: Shape (3, 1, 2, 2)
# W4_list = [[I],
#            [I],
#            [sz]]
# W4 = np.array(W4_list)

# print(" Basic operators and MPO tensors defined.")
# print(f"Shape of W1: {W1.shape}")
# print(f"Shape of W2: {W2.shape}")
# print(f"Shape of W3: {W3.shape}")
# print(f"Shape of W4: {W4.shape}\n")


# # --- 3. Contract the MPO Chain ---
# # We sequentially contract the tensors from left to right
# print("Contracting MPO chain: W1 @ W2 @ W3 @ W4 ...")
# M_temp = contract_mpo_pair(W1, W2)
# M_temp = contract_mpo_pair(M_temp, W3)
# M_mpo_tensor = contract_mpo_pair(M_temp, W4)

# # The final tensor has shape (1, 1, 16, 16). We squeeze it to get the 16x16 matrix.
# M_mpo = M_mpo_tensor.squeeze()
# print("Final MPO-derived matrix (first 4x4 block):\n", M_mpo[:4, :4].real)
# print("-" * 40)


# # --- 4. Construct the Exact Matrix for Verification ---
# print("⚙️  Constructing exact matrix using Kronecker products for verification...")
# # M = sz*sz*I*I + I*sz*sz*I + I*I*sz*sz
# M1 = np.kron(sz, np.kron(sz, np.kron(I, I)))
# M2 = np.kron(I, np.kron(sz, np.kron(sz, I)))

# M3 = np.kron(I, np.kron(I, np.kron(sz, sz)))
# M_exact = M1 + M2 + M3

# print("Final exact matrix (first 4x4 block):\n", M_exact[:4, :4].real)
# print("-" * 40)


# # --- 5. Compare the Results ---
# # np.allclose checks if two arrays are element-wise equal within a tolerance.
# are_matrices_equal = np.allclose(M_mpo, M_exact)

# print(f"Verification: Are the MPO and Exact matrices equal? \n  {are_matrices_equal}")

# if are_matrices_equal:
#     print("\n Success! The MPO construction correctly reproduced the full operator.")